In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 24.2 MB/s 
     |████████████████████████████████| 6.6 MB 66.3 MB/s 
     |████████████████████████████████| 120 kB 73.7 MB/s 


In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown --id 16xQuPZ8qgRVecIUK4Rc8QqfkoH7y8RAa
!unzip /content/bank_account_classifier_data.zip

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=16xQuPZ8qgRVecIUK4Rc8QqfkoH7y8RAa
To: /content/bank_account_classifier_data.zip
100% 41.6M/41.6M [00:01<00:00, 31.0MB/s]
Archive:  /content/bank_account_classifier_data.zip
   creating: bank_account_classifier_data/
  inflating: bank_account_classifier_data/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/bank_account_classifier_data/
  inflating: __MACOSX/bank_account_classifier_data/._.DS_Store  
  inflating: bank_account_classifier_data/bank_account_classifier_training_set.csv  
  inflating: bank_account_classifier_data/bank_account_classifier_test_set.csv  


In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer,BertModel

In [ ]:
df=pd.read_csv('/content/bank_account_classifier_data/bank_account_classifier_test_set.csv')

In [ ]:
df=df.sample(frac=1,random_state=10)

In [ ]:
df.isnull().sum()

labels                     0
full_bank_description      0
DB_NAME                    0
bankAccountBankName      297
bankAccountBranch        297
bankAmount                 0
lettering_code             0
payment_day                0
payment_month              0
payment_year               0
dtype: int64

In [ ]:
df.dropna(axis=0,inplace=True)
df.isnull().sum()

labels                   0
full_bank_description    0
DB_NAME                  0
bankAccountBankName      0
bankAccountBranch        0
bankAmount               0
lettering_code           0
payment_day              0
payment_month            0
payment_year             0
dtype: int64

In [ ]:
df.shape

(22263, 10)

In [ ]:
df['full_bank_description']= df['full_bank_description'].astype(str).values.tolist()
# df['labels']= df['labels'].astype(int)
# df['full_bank_description']= df['full_bank_description'].str.split("t", n =1, expand = False)

In [ ]:
# type(df['full_bank_description'][5])
# df['full_bank_description']=df['full_bank_description'].values.tolist()

In [ ]:
df['full_bank_description'][5]

'COMMISSIONS SUR REMISE CB NO 496924 DU 250521 99 '

In [ ]:
df=df.sample(1000)[['full_bank_description', 'labels']]

In [ ]:
# len(df)
# df.shape
# df['full_bank_description']=df['full_bank_description'].values
df

,full_bank_description,labels
7360,TPE 1505 CION,627
3353,TPE 1905 CION,627
8553,VRST REF14019A00 BILLET,530
468,190521 CB 1445 E.LECLERC 03AVERMES,401
22393,CHEQUE 6693548,627
...,...,...
14353,WEB LA CIGALE F00865 TAXES BURE FR763000400760...,401
9688,FRAIS VIREMENT NON SEPA,627
4999,COTISATION MENSUELLE CARTE 7535,627
1930,VIREMENT SEPA FACTURE 20210410 20210411 VIR SA...,411


In [ ]:
#If you’d like to select rows based on integer indexing, you can use the .iloc function.
#If you’d like to select rows based on label indexing, you can use the .loc function.
df.iloc[4]

full_bank_description    CHEQUE 6693548  
labels                                627
Name: 22393, dtype: object

In [ ]:

# sentences = df.full_bank_description.values
# label= df.labels.values
# type(sentences)

In [ ]:
le=preprocessing.LabelEncoder() 
df['labels']= le.fit_transform(df['labels'])

In [ ]:
# label = torch.tensor(label)
# label=F.one_hot(label, num_classes=57)
df['labels']=to_categorical(df['labels'])
# df['labels']=df['labels'].values.tolist()

In [ ]:
import numpy as np

In [ ]:
# train_target = torch.tensor(label.astype(np.float32))
# type(train_target)
# labels=df['labels'].values
# lab = torch.tensor(labels)
# label=F.one_hot(lab, num_classes=57)
# label[5].size()
df['labels'].unique()
# df['labels'].value_counts()

array([0., 1.], dtype=float32)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("ipuneetrathore/bert-base-cased-finetuned-finBERT")

In [ ]:
# encodings = tokenizer.batch_encode_plus(
#     df['full_bank_description'],
#     add_special_tokens=True,
#     max_length=256,
#     padding='max_length',
#     truncation=True,
#     return_attention_mask=True,
#     return_tensors='pt'
# )

In [ ]:
# encodings.input_ids[0]

In [ ]:
class BertDataset(torch.utils.data.Dataset):
  def __init__(self, df):
    self.df = df
    self.maxlen = 256
    self.tokenizer = BertTokenizer.from_pretrained("ipuneetrathore/bert-base-cased-finetuned-finBERT")

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    description = self.df['full_bank_description'].iloc[index].split()
    description = ' '.join(description)
    labels = int(self.df['labels'].iloc[index])
   # print(type(description))
   # print(len(description))
    encodings = self.tokenizer.encode_plus(
        description,
        add_special_tokens=True,
        max_length=self.maxlen,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
   # print("doe= process~")
    return {
        'input_ids': encodings.input_ids.flatten(),
        'attention_mask': encodings.attention_mask.flatten(),
        'labels': torch.tensor(labels, dtype=torch.long)
    }

In [ ]:
# df=[df['full_bank_description'].values,df['labels'].values]

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
len(train_df), len(test_df)

(800, 200)

In [ ]:
train_dataset = BertDataset(train_df)
valid_dataset = BertDataset(test_df)

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=5,
    shuffle=True
)

valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=5
)

In [ ]:
for batch in train_loader:
  print(batch['input_ids'].shape)
  print(batch['attention_mask'].shape)
  print(batch['labels'].shape)
  #print(type(batch['input_ids']))
  #print(type(batch['attention_mask']))
 # print((batch['labels']))
  break

torch.Size([5, 256])
torch.Size([5, 256])
torch.Size([5])


In [ ]:
import torch.nn as nn
from tqdm.auto import tqdm
import torch.optim as optim

In [ ]:
class tokenization(nn.Module):
  def __init__(self):
    super(tokenization, self).__init__()
    self.model = BertModel.from_pretrained("ipuneetrathore/bert-base-cased-finetuned-finBERT")
    self.drop0 = nn.Dropout(0.25)
    self.linear1 = nn.Linear(3072, 512)
    self.relu1 = nn.ReLU()
    self.drop1 = nn.Dropout(0.25)
    self.linear2 = nn.Linear(512, 57)
    self.relu2 = nn.ReLU()

  def forward(self, input_ids, attention_mask):
    outputs = self.model(input_ids, attention_mask)
    last_hidden_state = outputs[0]
    pooled_output = torch.cat(tuple([last_hidden_state[:, i] for i in [-4, -3, -2, -1]]), dim=-1)
    x = self.drop0(pooled_output)
    x = self.relu1(self.linear1(x))
    x = self.drop1(x)
    x = self.relu2(self.linear2(x))
    return x

In [ ]:
model = tokenization()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
epochs = 5

In [ ]:
for epoch in range(epochs):
  model.train()
  train_loop = tqdm(train_loader)
  for batch in train_loop:
    optimizer.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    #print(input_ids.shape)
    #print(attention_mask.shape)
    output = model(input_ids, attention_mask)
    loss = criterion(output, labels)
    loss.backward()
    nn.utils.clip_grad_norm(model.parameters(), max_norm=1.0)
    optimizer.step()
    train_loop.set_description(f"Training Epoch: {epoch}")
    train_loop.set_postfix(loss=loss.item())
  model.eval()
  valid_loop = tqdm(valid_loader)
  for batch in valid_loop:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    output = model(input_ids, attention_mask)
    loss = criterion(output, labels)
    valid_loop.set_description(f"Validation Epoch: {epoch}")
    valid_loop.set_postfix(loss=loss.item())

  0%|          | 0/160 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  


  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/160 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/160 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/160 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
# for epoch in range(epochs):

#     correct = 0
#     for i, (inputs,labels) in enumerate (train_loader):
#         ...
#         output = net(inputs)
#         ...
#         optimizer.step()

#         correct += (output == labels).float().sum()

#     accuracy = 100 * correct / len(trainset)
#     # trainset, not train_loader
#     # probably x in your case



TypeError: ignored